In [60]:
%load_ext autoreload
%autoreload 2

In [63]:
import netCDF4
import numpy as np
import os
import xarray as xr

import sys
sys.path.insert(0, '..')

from src.stream import get_stream_velocity

In [62]:
# load the NetCDF file (NetCDF4 library needed)
path_dataset = '../data/cmems_20200916_3.nc'
assert os.path.isfile(path_dataset)
cmems = xr.open_dataset(path_dataset)

In [54]:
cmems

<xarray.Dataset>
Dimensions:    (latitude: 1441, longitude: 4320, time: 3)
Coordinates:
    depth      float32 ...
  * latitude   (latitude) float32 -60.0 -59.916668 -59.833332 ... 59.916668 60.0
  * longitude  (longitude) float32 -180.0 -179.91667 ... 179.83333 179.91667
  * time       (time) datetime64[ns] 2020-09-16T12:00:00 ... 2020-09-18T12:00:00
Data variables:
    uo         (time, latitude, longitude) float32 ...
    vo         (time, latitude, longitude) float32 ...
Attributes:
    title:                              daily mean fields from Global Ocean P...
    easting:                            longitude
    northing:                           latitude
    history:                            2020/09/16 01:29:42 MERCATOR OCEAN Ne...
    source:                             MERCATOR PSY4QV3R1
    institution:                        MERCATOR OCEAN
    references:                         http://www.mercator-ocean.fr
    comment:                            CMEMS product
    Conventions:                        CF-1.4
    domain_name:                        GL12
    FROM_ORIGINAL_FILE__field_type:     mean
    field_date:                         2020-09-25 00:00:00
    field_julian_date:                  25835.0
    julian_day_unit:                    days since 1950-01-01 00:00:00
    forecast_range:                     9-day_forecast
    forecast_type:                      forecast
    bulletin_date:                      2020-09-16 00:00:00
    bulletin_type:                      operational
    FROM_ORIGINAL_FILE__longitude_min:  -180.0
    FROM_ORIGINAL_FILE__longitude_max:  179.91667
    FROM_ORIGINAL_FILE__latitude_min:   -80.0
    FROM_ORIGINAL_FILE__latitude_max:   90.0
    z_min:                              0.494025
    z_max:                              5727.917
    _CoordSysBuilder:                   ucar.nc2.dataset.conv.CF1Convention

In [55]:
# index the variable u, time 2020-09-16 and lon-lat (-10, 10) x (-20, 20)
u = cmems['uo'].sel(time='2020-09-16', longitude=slice(-10, 10), latitude=slice(-20, 20))
# extract the numpy array
u = u.data

(u < 0).sum()

52650

In [56]:
# index the variable v, time 2020-09-16 and lon-lat (-10, 10) x (-20, 20)
v = cmems['vo'].sel(time='2020-09-16', longitude=slice(-10, 10), latitude=slice(-20, 20))
# extract the numpy array
v = v.data

(v < 0).sum()

33164

In [59]:
np.concatenate((u, v)).shape

(2, 481, 241)

In [66]:
lat = (-10, 10)
lon = (-20, 20)
date = '2020-09-16'

get_stream_velocity(cmems, lat, lon, date)

array([[[-0.0512711 , -0.04760887, -0.03906369, ...,         nan,
                 nan,         nan],
        [-0.05005036, -0.04821924, -0.04394665, ...,         nan,
                 nan,         nan],
        [-0.0512711 , -0.05005036, -0.0469985 , ...,         nan,
                 nan,         nan],
        ...,
        [ 0.13611256,  0.16235846,  0.18494217, ...,         nan,
                 nan,         nan],
        [ 0.13000885,  0.15381329,  0.17212439, ...,         nan,
                 nan,         nan],
        [ 0.12573627,  0.14587848,  0.16052736, ...,         nan,
                 nan,         nan]],

       [[ 0.01953185,  0.02014222,  0.02136296, ...,         nan,
                 nan,         nan],
        [ 0.01281777,  0.01098666,  0.01281777, ...,         nan,
                 nan,         nan],
        [ 0.01098666,  0.00793481,  0.00732444, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.14770958,  0.15198217,  0.15808588, ...,  